# Calcium imaging

<div class="alert alert-block alert-info">


### ABOUT THIS LESSON 
---

Lesson time: 60 m   
Contributors: Davide Spalla, Melisa Maidana Capitan


**In this lesson you will learn:**
- point 1
- point 2 ...



***Note boxes templates***:

<div class="alert alert-block alert-info">

### GENERAL NOTE

Use this box for a general note on a topic that is lateral to the lessons's flow, but worth talikng about. 

<div class="alert alert-block alert-success"> 

### TIP  

Use this box for a tip or a small suggestion


<div class="alert alert-block alert-warning"> 

### WARNING  

Use this box for a warning about a common error or a difficult concept

## Introduction
---
 
General introduction

## Calcium imaging pipeline with CaImAn
---


### Summary images
---

In [35]:
#code

### Motion correction
---

### Source extraction
---


### Component evaluation
---


In [ ]:
# code

## Further steps
---


### Extracting spike counts with deconvolution
---


### Matching cells across sessions
---


<div class="alert alert-block alert-info">


### Key points 

- key point 1
- key point 2

<div class="alert alert-block alert-info">


### References and resources

**Books & papers**
* Book 1

**Websites & blogposts**
* syte one

**Software**
* library 1



## Exercises
You can find the exercises for this lessons in [notebook name](notebook_link)